### Import dataset

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
dataset_path = "/content/drive/My Drive/Colab Notebooks/garbage-dataset/images"

### Data Preprocessing

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define Image Size and Batch Size
img_size = 224  # Image size (MobileNetV2 requires 224x224)
batch_size = 32  # Number of images processed at once

# Data preprocessing & augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values (0-255 → 0-1)
    rotation_range=30,  # Rotate images up to 30 degrees
    width_shift_range=0.2,  # Shift images horizontally by 20%
    height_shift_range=0.2,  # Shift images vertically by 20%
    shear_range=0.2,  # Apply shear transformations
    zoom_range=0.2,  # Zoom in/out by 20%
    horizontal_flip=True,  # Flip images horizontally
    validation_split=0.2  # 80% train, 20% validation
)

# Load Training Data (80% of dataset)
train_data = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',  # Use 'binary' for Recyclable vs. Non-Recyclable
    subset='training'
)

# Load Validation Data (20% of dataset)
val_data = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Mapping of class names to numeric labels
print("Class Labels:", train_data.class_indices)  # {'nonrecyclable': 0, 'recyclable': 1}

Found 15827 images belonging to 2 classes.
Found 3956 images belonging to 2 classes.
Class Labels: {'Non Recyclable': 0, 'Recyclable': 1}


### Load Model (MobileNetV2)

In [11]:
#import required libraries
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

# Load MobileNetV2 without the top (classification) layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers = keep pretrained knowledge from ImageNet unchanged
base_model.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduce feature map to a single vector
x = Dense(128, activation='relu')(x)  # Fully connected layer
x = Dropout(0.5)(x)  # Prevent overfitting
output_layer = Dense(1, activation='sigmoid')(x)  # Binary classification → 1 output neuron with sigmoid

# Create the final model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model for binary classification
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Show model summary
model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise_BN 

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

### Train Model

In [ ]:
epochs = 10  # Number of training cycles (increase for better accuracy)
history = model.fit(train_data, validation_data=val_data, epochs=epochs)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 6584s 13s/step - accuracy: 0.9043 - loss: 0.2303 - val_accuracy: 0.5597 - val_loss: 1.5779
Epoch 2/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 1198s 2s/step - accuracy: 0.9507 - loss: 0.1333 - val_accuracy: 0.6024 - val_loss: 1.2749
Epoch 3/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 1226s 2s/step - accuracy: 0.9599 - loss: 0.1105 - val_accuracy: 0.6062 - val_loss: 1.1159
Epoch 4/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 1195s 2s/step - accuracy: 0.9601 - loss: 0.1094 - val_accuracy: 0.5602 - val_loss: 1.5788
Epoch 5/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 1223s 2s/step - accuracy: 0.9597 - loss: 0.1007 - val_accuracy: 0.6206 - val_loss: 1.0821
Epoch 6/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 1194s 2s/step - accuracy: 0.9670 - loss: 0.0884 - val_accuracy: 0.6279 - val_loss: 1.1789
Epoch 7/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 1224s 2s/step - accuracy: 0.9686 - loss: 0.0827 - val_accuracy: 0.5837 - val_loss: 1.6716
Epoch 8/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 1199s 2s/step - accuracy: 0.9672 - loss: 0.0829 -


### Save Trained Model

In [ ]:
import keras.saving
keras.saving.save_model(model, "/content/drive/My Drive/Colab Notebooks/TrainedModelRecyclable.keras")
print("Model saved successfully!")


Model saved successfully!


### Testing Model

In [13]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load a test image
img_path = "/content/drive/My Drive/Colab Notebooks/biowastetest.jpg"  # Change this path
img = image.load_img(img_path, target_size=(224, 224))  # Resize image

# Convert image to an array and normalize
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Make prediction
predictions = model.predict(img_array)

# Interpret the result
predicted_class = int(predictions[0][0] > 0.5)  # 0 = Non-recyclable, 1 = Recyclable
labels = ["Non-Recyclable", "Recyclable"]
print(f"Predicted class: {labels[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Predicted class: Recyclable
